In [47]:
!pip install -U -q "google"
!pip install -U -q "google.genai"

import os
from google.colab import userdata
from google.colab import drive
os.environ["GEMINI_API_KEY"] = userdata.get("GOOGLE_API_KEY")

drive.mount("/content/drive")
# Please ensure that uploaded files are available in the AI Studio folder or change the working folder.
os.chdir("/content/drive/MyDrive/Google AI Studio")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [49]:
# To run this code you need to install the following dependencies:
# pip install google-genai

import base64
import os
from google import genai
from google.genai import types
import pandas as pd
import time

# Your existing generate function modified to accept text input
def generate(text):
    client = genai.Client(
        api_key=os.environ.get("GEMINI_API_KEY"),
    )

    model = "gemini-2.5-flash"
    contents = [
        types.Content(
            role="user",
            parts=[
                types.Part.from_text(text=text),
            ],
        ),
    ]
    generate_content_config = types.GenerateContentConfig(
        temperature=0.1,
        thinking_config = types.ThinkingConfig(
            thinking_budget=0,
        ),
        response_mime_type="text/plain",
        system_instruction=[
            types.Part.from_text(text="""## Task:
Determine whether the user provided social media post is related to **any real disaster/threat or not**. Options are: 1 (text: \"a disaster\") else 0 (text: \"not a disaster\")

## Examples:
I actually posted this by accident to twitter but have decided this can stay = 0
Why are Puerto Ricans refusing to go home? Aside from the 5.4 aftershock just now, the ground in southwest Puerto Rico has… = 1
I’m here on Hwy 278 near Thaxton where a home is completely engulfed in flames... Thaxton and Pontotoc emergency crews are… = 1
Each financial year money is allocated for disaster management, but we still raise paybills during famine and floods #UhuruAddress = 0
Telangana: Section 144 has been imposed in Bhainsa from January 13 to 15, after clash erupted between two groups on January 12. Po… = 1
Help for Animal Fire Victims Premier Annastacia Palaszczuk has announced grants for wildlife carers inundated with… https://t.co/x0sedOKDlg... = 0
#PUERTORICO: EVERTEC, yesterday announced that the Company will donate $50,000 for earthquake relief in Puerto Rico. = 0


## Output Format <only the corresponding number>:
1 or 0
"""),
        ],
    )

    try:
        for chunk in client.models.generate_content_stream(
            model=model,
            contents=contents,
            config=generate_content_config,
        ):
            return int(chunk.text.strip())  # Convert response to integer and return
    except Exception as e:
        print(f"Error processing text: {text[:50]}... Error: {str(e)}")
        return None  # Return None if there's an error

if __name__ == "__main__":
    # Load your dataset
    df = pd.read_csv("/content/drive/MyDrive/ML project - Nurshat/tweets.csv", usecols=['text','target'])

    df = df.sample(n=30).copy()

    # Create a new column for model predictions
    df['prediction'] = None
    i=0
    # Process each row
    for idx, row in df.iterrows():
        try:
            # Get the prediction from Gemini
            prediction = generate(row['text'])
            df.at[idx, 'prediction'] = prediction
            i=i+1
            # Print progress
            print(f"Processed {i}/{len(df)} - Text: {row['text']}... - Target: {row['target']} - Prediction: {prediction}")

            # Add delay to avoid hitting rate limits
            if {idx+1}==10:
              time.sleep(15)
            else:
              time.sleep(5)

        except Exception as e:
            print(f"Error processing row {idx}: {str(e)}")
            continue

    # Calculate accuracy (only for rows where we got a prediction)
    valid_predictions = df.dropna(subset=['prediction'])
    if len(valid_predictions) > 0:
        accuracy = (valid_predictions['target'] == valid_predictions['prediction']).mean()
        print(f"\nModel Accuracy: {accuracy:.2%}")
    else:
        print("No valid predictions were obtained.")

    # Save results to a new CSV
    output_path = "/content/drive/MyDrive/ML project - Nurshat/tweets_with_predictions.csv"
    df.to_csv(output_path, index=False)
    print(f"\nResults saved to {output_path}")

Processed 1/30 - Text: That sandwich was bioterrorism.... - Target: 0 - Prediction: 0
Processed 2/30 - Text: Graffiti and damage around CS Lewis Sq. Some folk need to wise up. I’ve reported to and hopefully cleaned so…... - Target: 0 - Prediction: 0
Processed 3/30 - Text: date a girl who says: - drive safe - i'm proud of you - Christmas is a trap where you’re guilted to feel closer to your f…... - Target: 0 - Prediction: 0
Processed 4/30 - Text: #DearNamjoon I’m 💯% back from injury rehab. I still listen to PERSONA when I lift because you motivate me so much!🏋🏻‍♀️…... - Target: 0 - Prediction: 0
Processed 5/30 - Text: On the flip side, if we compare the 2003 low in pedestrian/cycling fatalities to today (in the US), we have… https://t.co/y9urqaVy9c... - Target: 0 - Prediction: 0
Processed 6/30 - Text: Run your own race. If your desire to cross that threshold is strong,invest considerable time in learning financial…... - Target: 0 - Prediction: 0
Processed 7/30 - Text: Giving birth is th

In [50]:
test_sentence = "There is a cyclone in Florida"
import spacy
nlp = spacy.load('en_core_web_sm')
doc = nlp(test_sentence)
locations = [ent.text for ent in doc.ents if ent.label_ == 'GPE']
print("Disaster Locations:", locations)

Disaster Locations: ['Florida']


In [51]:
def test_single_sentence(sentence):
    test_prediction = generate(sentence)  # Directly get 0 or 1 from Gemini
    print(f'Test sentence: "{sentence}" is {"a disaster" if test_prediction == 1 else "not a disaster"}')

# Example usage
if __name__ == "__main__":
    test_sentence = "My life is a cyclone"
    test_single_sentence(test_sentence)

Test sentence: "My life is a cyclone" is not a disaster
